# Simulación Flujo de Stokes

Se intentará resolver la ecuación homogénea con condición de frontera de Dirichlet, considerando $\mu=1$.

Primero, partimos importando las librerías que utilizaremos. Notar que ```bempp``` corresponde a una modificación de la librería original que incluya las nuevas implementaciones para el flujo de Stokes.

In [1]:
import bempp.api
import numpy as np

Se sitúa una esfera en el origen con radio 1:

In [2]:
grid = bempp.api.shapes.sphere(h=0.1)

A continuación, se define el espacio a utilizar. En nuestro caso, utilizaremos funciones P1 debido a que nuestra incógnita corresponde a la función de Neumann y que el costo computacional no es significativo para el caso que estamos estudiando.

In [3]:
p1_space = bempp.api.function_space(grid, "P", 1)

Ahora, definimos a los operadores. En nuestro caso, solo necesitamos al single layer y este se encuentra programado por coordenadas, por lo que armamos un bloque de capas:

In [4]:
slp1 = bempp.api.operators.boundary.stokes.single_layer(
    p1_space, p1_space, p1_space, opt_layer=0
)
slp2 = bempp.api.operators.boundary.stokes.single_layer(
    p1_space, p1_space, p1_space, opt_layer=1
)
slp3 = bempp.api.operators.boundary.stokes.single_layer(
    p1_space, p1_space, p1_space, opt_layer=2
)
slp4 = bempp.api.operators.boundary.stokes.single_layer(
    p1_space, p1_space, p1_space, opt_layer=3
)
slp5 = bempp.api.operators.boundary.stokes.single_layer(
    p1_space, p1_space, p1_space, opt_layer=4
)
slp6 = bempp.api.operators.boundary.stokes.single_layer(
    p1_space, p1_space, p1_space, opt_layer=5
)
slp7 = bempp.api.operators.boundary.stokes.single_layer(
    p1_space, p1_space, p1_space, opt_layer=6
)
slp8 = bempp.api.operators.boundary.stokes.single_layer(
    p1_space, p1_space, p1_space, opt_layer=7
)
slp9 = bempp.api.operators.boundary.stokes.single_layer(
    p1_space, p1_space, p1_space, opt_layer=8
)

A = bempp.api.GeneralizedBlockedOperator(
    [[slp1, slp2, slp3], [slp4, slp5, slp6], [slp7, slp8, slp9]]
)

Definimos la función de grilla en la esfera, representando a la parte de Dirichlet:

In [5]:
@bempp.api.real_callable
def dirichlet_data(x, n, domain_index, result):
    result[0] = 0


dirichlet_fun = bempp.api.GridFunction(p1_space, fun=dirichlet_data)

Construimos el lado derecho de la ecuación integral de borde (problema interior)

$$(\tfrac12\mathsf{Id}+\mathsf{K})u.$$

In [6]:
rhs = [dirichlet_fun, dirichlet_fun, dirichlet_fun]

Resolvemos el sistema lineal mediante una descomposición LU directa:

In [10]:
Tu = bempp.api.linalg.lu(A, rhs)

Definimos puntos en los que evaluaremos la solución:

In [11]:
n_grid_points = 150
plot_grid = np.mgrid[-3 : 3 : n_grid_points * 1j, -3 : 3 : n_grid_points * 1j]
points = np.vstack(
    (plot_grid[0].ravel(), plot_grid[1].ravel(), np.zeros(plot_grid[0].size))
)

Utilizamos las formulas de representación para evaluar la velocidad y la presión en los puntos recién definidos:

In [13]:
# Velocidad:
slp1_pot = bempp.api.operators.potential.stokes.single_layer(
    p1_space, points, opt_layer=0
)
slp2_pot = bempp.api.operators.potential.stokes.single_layer(
    p1_space, points, opt_layer=1
)
slp3_pot = bempp.api.operators.potential.stokes.single_layer(
    p1_space, points, opt_layer=2
)
slp4_pot = bempp.api.operators.potential.stokes.single_layer(
    p1_space, points, opt_layer=3
)
slp5_pot = bempp.api.operators.potential.stokes.single_layer(
    p1_space, points, opt_layer=4
)
slp6_pot = bempp.api.operators.potential.stokes.single_layer(
    p1_space, points, opt_layer=5
)
slp7_pot = bempp.api.operators.potential.stokes.single_layer(
    p1_space, points, opt_layer=6
)
slp8_pot = bempp.api.operators.potential.stokes.single_layer(
    p1_space, points, opt_layer=7
)
slp9_pot = bempp.api.operators.potential.stokes.single_layer(
    p1_space, points, opt_layer=8
)

u_evaluated1 = slp1_pot * Tu[0] + slp2_pot * Tu[1] + slp3_pot * Tu[2]
u_evaluated2 = slp4_pot * Tu[0] + slp5_pot * Tu[1] + slp6_pot * Tu[2]
u_evaluated3 = slp7_pot * Tu[0] + slp8_pot * Tu[1] + slp9_pot * Tu[2]

In [14]:
print(u_evaluated1)
print(u_evaluated2)
print(u_evaluated3)

[[0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]]


In [15]:
# Presión:
slp10_pot = bempp.api.operators.potential.stokes.single_layer(
    p1_space, points, opt_layer=9
)
slp11_pot = bempp.api.operators.potential.stokes.single_layer(
    p1_space, points, opt_layer=10
)
slp12_pot = bempp.api.operators.potential.stokes.single_layer(
    p1_space, points, opt_layer=11
)

p_evaluated = slp10_pot * Tu[0] + slp11_pot * Tu[1] + slp12_pot * Tu[2]

In [16]:
print(p_evaluated)

[[0. 0. 0. ... 0. 0. 0.]]
